In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
sys.path.append('..')

In [ ]:
import pandas as pd
import sqlite3 as db
from langdetect import detect
import chess
import chess.pgn
import io
import utils.utils as utils
import nltk
import gensim
import re

In [5]:
con = db.connect("./chess.db")
games = pd.read_sql_query("SELECT * FROM gameknot_games WHERE language == 'en'", con)
games

,id,pgn,language
0,44,"[Event ""alygally&#039;s mini-tournament""]\n[Da...",en
1,62,"[Event ""League division D2""]\n[Date ""08-Jun-06...",en
2,64,"[Event ""Let&#039;s play chess""]\n[Date ""25-May...",en
3,66,"[Event ""easy street""]\n[Date ""26-Jun-06""]\n[Wh...",en
4,70,"[Event ""001 mini-tournament""]\n[Date ""31-May-0...",en
...,...,...,...
12243,70354,"[Event ""A question of queens...""]\n[Date ""14 S...",en
12244,70355,"[Event ""50th World Open U1800 Round 9 '22""]\n[...",en
12245,70359,"[Event ""Team match""]\n[Date ""2022.06.26""]\n[Wh...",en
12246,70364,"[Event ""Team match""]\n[Date ""2022.06.30""]\n[Wh...",en


In [6]:
# english_annotated_moves = utils.games_to_moves(games)
# english_annotated_moves.head(5)

In [5]:
# english_annotated_moves.applymap(str).to_sql("english_annotated_moves", con, index=True, if_exists="replace")

In [6]:
english_annotated_moves = pd.read_sql_query("SELECT * FROM english_annotated_moves", con)
english_annotated_moves.drop('index', inplace=True, axis=1)
# english_annotated_moves["sentiment"] = -1
english_annotated_moves
display(english_annotated_moves)

,position,move,comment,halfmove_number,game_id,sentiment,auto_sentiment
0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,e2e4,This is my first gameknot game against someone...,1,44,-1,0
1,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,b1c3,"I've been playing the Vienna Gambit as white, ...",3,44,-1,0
2,rnbqkbnr/pppp1ppp/8/4p3/4P3/2N5/PPPP1PPP/R1BQK...,f8c5,Minor disappointment.,4,44,-1,0
3,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2NP4/PPP2PP...,f2f4,"My idea here is to expand on the kingside, dri...",9,44,-1,0
4,r1bqk2r/ppp2ppp/3p1n2/n1b1pP2/2B1P3/2NP4/PPP3P...,d1f3,"Maybe this isn't the greatest plan, since with...",13,44,-1,0
...,...,...,...,...,...,...,...
326634,rnb5/p2p1P1N/7p/1pR5/6P1/4k3/2P1B3/3Q1RK1 b - ...,b5b4,Pawn,86,70387,-1,0
326635,rnb5/p2p1P1N/7p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 w -...,f7f8q,Check mate in two moves,87,70387,-1,0
326636,rnb2Q2/p2p3N/7p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 b -...,d7d6,Pawn,88,70387,-1,0
326637,rnb2Q2/p6N/3p3p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 w -...,f8e8,Check,89,70387,-1,0


In [7]:
english_annotated_moves.loc[english_annotated_moves.comment.str.match("^[!]{1,2}[ ]"), 'sentiment'] = 1
english_annotated_moves.loc[english_annotated_moves.comment.str.match("^[!]{1,2}[ ]"), 'auto_sentiment'] = 1
# english_annotated_moves[english_annotated_moves.comment.str.match("^[!]{1,2}[ ]")]
english_annotated_moves

,position,move,comment,halfmove_number,game_id,sentiment,auto_sentiment
0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,e2e4,This is my first gameknot game against someone...,1,44,-1,0
1,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,b1c3,"I've been playing the Vienna Gambit as white, ...",3,44,-1,0
2,rnbqkbnr/pppp1ppp/8/4p3/4P3/2N5/PPPP1PPP/R1BQK...,f8c5,Minor disappointment.,4,44,-1,0
3,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2NP4/PPP2PP...,f2f4,"My idea here is to expand on the kingside, dri...",9,44,-1,0
4,r1bqk2r/ppp2ppp/3p1n2/n1b1pP2/2B1P3/2NP4/PPP3P...,d1f3,"Maybe this isn't the greatest plan, since with...",13,44,-1,0
...,...,...,...,...,...,...,...
326634,rnb5/p2p1P1N/7p/1pR5/6P1/4k3/2P1B3/3Q1RK1 b - ...,b5b4,Pawn,86,70387,-1,0
326635,rnb5/p2p1P1N/7p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 w -...,f7f8q,Check mate in two moves,87,70387,-1,0
326636,rnb2Q2/p2p3N/7p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 b -...,d7d6,Pawn,88,70387,-1,0
326637,rnb2Q2/p6N/3p3p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 w -...,f8e8,Check,89,70387,-1,0


In [8]:
english_annotated_moves.loc[english_annotated_moves.comment.str.match("^[?]{1,2}[ ]"), 'sentiment'] = 0
english_annotated_moves.loc[english_annotated_moves.comment.str.match("^[?]{1,2}[ ]"), 'auto_sentiment'] = 1
english_annotated_moves.applymap(str).to_sql("english_annotated_moves", con, index=True, if_exists="replace")

326639

In [9]:
# english_annotated_moves.loc[english_annotated_moves.comment.str.match("^[!]{1,2}[ ]")].comment = 1
english_annotated_moves[english_annotated_moves['comment'].str.match("^[!]{1,2}[ ]")]
# english_annotated_moves['comment'].str.match("^[!]{1,2}[ ]")

,position,move,comment,halfmove_number,game_id,sentiment,auto_sentiment
59,r7/pp1bkprp/2n1pN2/4P3/3P4/P7/5KPP/3R1B1R w - ...,f1d3,!! brilliant move it gives Black the d4 pawn b...,43,70,1,1
61,r7/pp1bkpr1/2n1pN1p/4P3/3P4/P2B4/5KPP/3R3R w -...,h2h4,!! this move is really fantastic. main job of ...,45,70,1,1
65,2r5/pp2kpr1/4pN1p/4P3/4R2P/P7/5KP1/7R w - - 1 27,e4d4,! wow White does not care his g2 pawn do u see...,53,70,1,1
71,8/5p2/8/R3pk1p/7P/P7/5KP1/8 w - - 1 37,f2f3,! stoping king,73,70,1,1
110,r2qk2r/pp3pp1/2p1bnnp/8/3PP3/P1PB1N2/2Q2PPP/R1...,e4e5,! Both the Nf6 and the Ng6 are attacked - Blac...,25,111,1,1
...,...,...,...,...,...,...,...
326279,r1bq1rk1/pp2bppp/2n1pn2/2pp4/3P4/PP1BPN2/1BP2P...,b7b6,! good move. Black fianchetto his Queen's bish...,16,70342,1,1
326290,r4rk1/1b4pp/pnn1p1q1/2ppPp2/2P2P2/P2BPN1Q/1B1R...,c6a5,! It was here I seen the game shift towards Bl...,42,70342,1,1
326321,8/p2kn1pp/5p2/2Pr4/3N4/4KP1P/P5P1/2R5 b - - 6 36,e7f5,! Allowing black to secure a draw. I thought m...,72,70343,1,1
326356,2k1r3/1ppb2p1/p2q1p2/2p4r/P3Pp1p/1PQP1N1P/2P2P...,a1d1,! Time remaining 46/90 minutes.,49,70346,1,1
